# 자동차 연비 예측하기: 회귀

### 텐서플로와 다른 라이브러리 임포트

In [0]:
# 산점도 행렬을 그리기 위해 seaborn 패키지를 설치합니다
!pip install seaborn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [0]:
# 런타임에서 할당하는데 필요한 양만큼의 GPU 메모리를 할당
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

### Auto MPG 데이터셋 로딩

In [0]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

In [0]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                          na_values = "?", comment='\t',
                          sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

### 데이터 전처리

#### Q. 누락데이터 확인해서 제거하시오. (Hint : 누락 데이터 확인은 dataset.isna()으로 확인)

In [0]:
# 누락 데이터 확인은 dataset.isna()로 확인
#your code

#### Origin을 이용해서 One-Hot Vector 형태의 세개의 컬럼을 생성 ( 1은 USA, 2는 Europe, 3은 Japan)

In [0]:
origin = dataset.pop('Origin') # 1은 USA, 2는 Europe, 3은 Japan
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

#### Q. 데이터셋을 훈련 세트와 테스트 세트로 분할하시오. (Hint : dataset.sample(), dataset drop() 사용)

In [0]:
train_dataset = #your code
test_dataset = #your code

#### 산점도 행렬로 데이터 분포 및 상관성 조사

In [0]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")

#### 통계 확인

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

#### Q. 훈련 세트와 테스트 세트의 레이블 분리하시오. (Hint : dataset.pop() 사용)

In [0]:
# dataset.pop() 사용
train_labels = #your code
test_labels = #your code

####  데이터 정규화 
표준정규분포 정규화 사용 $z = \frac{X - \mu}{\sigma} \sim \mathcal{N}(0,1)$

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

### 모델

#### Q. 모델을 생성하는 build_model() 함수를 정의하시오.

In [0]:
def build_model():
  # input len(train_dataset.keys()) 
  # hidden 64, hidden 64, output 1
  model = keras.Sequential(#your code)

  # RMSProp으로 Optimizer 생성, learning rate = 0.001
  optimizer = #your code

  # loss, oprimizer, metric 설정
  # metric은 'mean_absolute_error’, 'mean_squared_error'로 설정
  model.compile(#your code)
  return model

In [0]:
model = build_model()

In [0]:
model.summary()

In [0]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

### 모델 훈련

#### Q. 모델을 훈련시키기 위해 model.fit() 함수를 완성하시오.
1. PrintDot() Callback 추가
2. validation_split을 이용해서 validation set은 training set의 20%로 설정
3. verbose=0로 설정해서 출력 없애기

In [0]:
# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 1000

# 1. PrintDot() Callback 추가
# 2. validate_split을 이용해서 validation set은 training set의 20%로 설정
# 3. verbose=0로 설정해서 출력 없애기
history = model.fit(#your code)

### 훈련 과정 시각화

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [0]:
import matplotlib.pyplot as plt

def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure(figsize=(8,12))

  plt.subplot(2,1,1)
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.subplot(2,1,2)
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()

plot_history(history)

### EarlyStopping
https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping 참고
#### Q. 훈련 시 early stopping할 수 있도록 callback을 추가하시오.

In [0]:
model = build_model()

# keras.callbacks.EarlyStopping 사용
# patience : 성능 향상을 체크할 에포크 횟수
early_stop = # your code

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [0]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("테스트 세트의 평균 절대 오차: {:5.2f} MPG".format(mae))

## 예측

#### 예측 테스트 결과와 실제 레이블과의 상관 관계 그래프

In [0]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

#### 오차 분포 히스토그램

In [0]:
# 오차의 분포
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")